Dopo aver calcolato il pairwise matching con **Record Linkage Toolkit**, è necessario confrontare i risultati con un metodo alternativo. Le opzioni disponibili sono:  

### **1️⃣ DeepMatcher**  
DeepMatcher è un modello basato su **reti neurali classiche**, ottimizzato per il record linkage su dati strutturati.  

✅ Richiede meno dati per il training  
✅ Facile da configurare  
✅ Funziona bene con dati tabellari strutturati (es. numeri, date, nomi)  

📌 **Quando usarlo?**  
Se i dati sono ben strutturati e vogliamo un approccio più rapido senza entrare nei modelli di NLP avanzati.  

🔗 **Repo:** [DeepMatcher](https://github.com/anhaidgroup/deepmatcher)  

---

### **2️⃣ Ditto**  
Ditto utilizza **modelli di trasformazione del linguaggio (NLP)** come BERT per catturare la semantica dei dati.  

✅ Potente per dati testuali rumorosi (es. descrizioni, indirizzi)  
✅ Richiede meno feature engineering rispetto a DeepMatcher  
✅ Utilizza **fine-tuning** su un transformer pre-addestrato  

📌 **Quando usarlo?**  
Se il dataset contiene molte informazioni testuali e vogliamo sfruttare l'intelligenza artificiale avanzata.  

🔗 **Repo:** [Ditto](https://github.com/megagonlabs/ditto)  

---

### **3️⃣ EMT (Entity Matching Transformer)**  
EMT è una variante di Ditto, più **efficiente e leggera**, ottimizzata per dataset di grandi dimensioni.  

✅ Simile a Ditto, ma con meno overhead computazionale  
✅ Adatto a dataset misti (dati strutturati e testuali)  
✅ Richiede meno risorse rispetto a Ditto  

📌 **Quando usarlo?**  
Se Ditto è troppo pesante e vogliamo un compromesso tra efficienza e accuratezza.  

🔗 **Repo:** [EMT](https://github.com/brunnurs/entity-matching-transformer)  

---

### **💡 Scelta del Metodo**  
1️⃣ Se vogliamo un metodo rapido e semplice **→ DeepMatcher**  
2️⃣ Se vogliamo massima accuratezza **→ Ditto**  
3️⃣ Se Ditto è troppo pesante **→ EMT**  

🔹 **Prossimo passo:** iniziamo testando **DeepMatcher**, poi valuteremo se passare a un modello basato su transformer come **Ditto** o **EMT**. 🚀  

## Implementazione di DeepMatcher  

In questa sezione utilizziamo **DeepMatcher**, un modello basato su reti neurali per il record linkage.  
L'obiettivo è confrontare i risultati ottenuti con Record Linkage Toolkit e verificare quale approccio funziona meglio.  

---

### **1️⃣ Formattazione del Dataset**  

DeepMatcher richiede che i dati siano strutturati in un file CSV con le seguenti colonne:  

- **id**: Identificativo univoco della coppia  
- **left_***: Attributi della prima azienda  
- **right_***: Attributi della seconda azienda  
- **label**: **1** se le aziende sono un match, **0** altrimenti  

📌 **Nota:** Dobbiamo creare manualmente le etichette di match/non-match, basandoci sulla nostra ground truth.  

---

### **2️⃣ Installazione di DeepMatcher**  

Prima di procedere, installiamo le librerie necessarie: 

In [9]:
!pip install torch

In [1]:
import torch
print(torch.cuda.is_available())  # Deve restituire True se la GPU è attiva

False


In [ ]:
import deepmatcher as dm
import torch
import torchtext

print("DeepMatcher, Torch, and TorchText are successfully imported.")

OSError: [WinError 127] Impossibile trovare la procedura specificata

# **NON MI FUNZIONA PORCODIO**

## **Proviamo con DITTO**
# Installazione e Utilizzo di Ditto per il Record Linkage

**Ditto** è una libreria Python semplice ed efficiente per il *record linkage* che permette di trovare e abbinare record simili da due dataset. In questa guida, mostreremo come installarla e usarla per eseguire il matching tra due dataset.

## Installazione

### 1. Installazione con pip

Per installare Ditto, esegui il seguente comando:

In [11]:
!pip install ditto

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ditto: filename=ditto-1.2.4-py3-none-any.whl size=13921 sha256=9f34f9dbf6278989fe978f8c68980770ebafe020bd78b8bca16d1eeff1964797
  Stored in directory: c:\users\matte\appdata\local\pip\cache\wheels\75\e3\03\f60df6f0b06c9b3c2dec01af270c80cc0d1001648a899217b0
Successfully built ditto


In [12]:
import ditto

In [14]:
import ditto

# Define dataset
dataset = [
    {"id": 1, "name": "Company A", "city": "New York"},
    {"id": 2, "name": "Company A", "city": "New York"},
    {"id": 3, "name": "Company B", "city": "San Francisco"},
    {"id": 4, "name": "Company C", "city": "Chicago"}
]

# Create DittoMatcher
matcher = ditto.DittoMatcher()

# Perform matching on the dataset (same dataset as both arguments)
matches = matcher.match(dataset, dataset)

# Display results
for match in matches:
    print(match)


AttributeError: module 'ditto' has no attribute 'DittoMatcher'